In [ ]:
# Libraries install ho rahi hain
!pip install -q langchain langchain-community chromadb sentence-transformers pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF ko read karna
loader = PyPDFLoader("test.pdf")
data = loader.load()

print(f"Total Pages: {len(data)}")
print(f"Pehle page ka sample: {data[0].page_content[:200]}")

Total Pages: 7
Pehle page ka sample: V erifyNo w — A Quick 
Onboar ding Guide
 St ep 1: Sign Up & Log In 
Go t o Message C entral Signupu.lS
Cre a t e y our f ree accountu.lS
Y ou’ll inst antly get f ree credits t o t est O TP s
  Tip: U


In [ ]:
!pip install -U langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1000 characters ka ek chunk, aur 200 characters overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

print(f"Total Chunks: {len(chunks)}")

Total Chunks: 11


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Embedding model download hoga (Free & Open Source)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Vector Database banana
vector_db = Chroma.from_documents(documents=chunks, embedding=embedding_model)

print("Vector Database taiyaar hai!")

/tmp/ipython-input-3061317497.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Vector Database taiyaar hai!


In [ ]:
query = "What is the main topic of this document?" # Apne PDF ke hisaab se pucho
docs = vector_db.similarity_search(query, k=2)

for i, doc in enumerate(docs):
    print(f"\nChunk {i+1}:\n", doc.page_content)


Chunk 1:
 V erifyNo w — A Quick 
Onboar ding Guide
04
Send O TP
T o sendOtp  on a mobile number belo w are the request paramet ers. The authen tic a tion t ok en is 
required t o send O TP which is genera t ed b y the genera t ed t ok en API (which y ou c an .etind abo v e in 
Introduction section).
  API P aramet er T ype Manda t ory?
authT ok en String Y es
R equest URL P a th:
A successful response will return a 200 st a tus code.
POST
/verification/ /send v3
R equest URL P aramet ers:
Field T ype Manda t ory? Descrip tion
cust omerId String y es C ountry code 
o tpLength Int eger no Send a number 
betw een 4 and 8. 
Def ault is 4
mobileNumber String y es Mobile number 
f or single t e xt
.endao wT ype String y es W e send O TP 
using multiple 
mediums lik e 
SMS ,  W ha tsApp ,  
email ,  et c.  F or 
no w ,  use SMS 
only

Chunk 2:
 &scope=NEW&country=91&email=test%40messagecentral.com' rh?/
--header 'accept: */*'
https://cpaas.messagecentral.com/auth/v1/
authentication/token?
 N

In [ ]:
!pip install -q langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain_classic

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# A. API Key Setup
os.environ["GROQ_API_KEY"] = ""

# B. LLM Initialize (Groq is super fast!)
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

# C. Professional Prompt Template (Workshop mein ye dikhana imp hai)
system_prompt = (
    "You are an expert assistant. Use the provided context to answer the user's question. "
    "If the answer isn't in the context, just say you don't know. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# D. The Chain Building (Modern LCEL Way)
# Pehle 'Documents' ko 'Prompt' ke saath jodne wali chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Phir 'Retriever' aur 'Documents Chain' ko milakar final RAG chain
rag_chain = create_retrieval_chain(vector_db.as_retriever(), combine_docs_chain)

# E. Execution
response = rag_chain.invoke({"input": "What are the main points of this document?"})

print("--- AI ANSWER ---")
print(response["answer"])

--- AI ANSWER ---
The main points of this document are:

1. **Sign Up and Log In**: Create a free account on Message Central Signup to get free credits for testing OTPs.
2. **API Integration**: Integrate the VerifyNow API into your application using ready-made code snippets or a video tutorial.
3. **API Parameters**: Understand the required API parameters, such as `u`, `type`, and `authToken`.
4. **Validate OTP**: Use the `validateOtp` method to validate a one-time password (OTP) for customers.
5. **Authentication**: Obtain an authentication token by calling the `/auth/v1/authentication/token` endpoint with required parameters such as `customerId`, `key`, and `scope`.
6. **API Endpoints**: Use the provided API endpoints, such as `/verification/validateOtp/v3`, to interact with the VerifyNow API.
7. **cURL Examples**: Use cURL commands to test API requests, such as validating an OTP or obtaining an authentication token.

These points provide a general overview of how to get started with

In [ ]:
# Ye dikhayega ki database se kaunse chunks uthaye gaye
for i, doc in enumerate(response["context"]):
    print(f"\n--- Chunk {i+1} (Source: {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:200] + "...")


--- Chunk 1 (Source: 0) ---
V erifyNo w — A Quick 
Onboar ding Guide
 St ep 1: Sign Up & Log In 
Go t o Message C entral Signupu.lS
Cre a t e y our f ree accountu.lS
Y ou’ll inst antly get f ree credits t o t est O TP s
  Tip: U...

--- Chunk 2 (Source: 5) ---
V erifyNo w — A Quick 
Onboar ding Guide
05
V alida t e O TP
The  v alida t eOtp method is a REST API endpoint f or v alida ting a one-time pass w ord ( O TP) f or 
cust omers.
R equest He ader T ype ...

--- Chunk 3 (Source: 6) ---
V erifyNo w — A Quick 
Onboar ding Guide
06
cURL
kJ66
kJ61
kJ6x
kJ6j
5
curl --loc a tion 'https://
&v erikJ4=c a tionId=29 49&code=147 6' kJWE
--he ader 'authT ok en: 
eyJhbGciOiJIUzUxMiJ9 .eyJzdWIiOi...

--- Chunk 4 (Source: 1) ---
V erifyNo w — A Quick 
Onboar ding Guide
This API returns a t ok en tha t must be included in all subsequent c alls. The authenti c a tion t ok en is 
required t o v alida t e the user and must be inc...
